In [1]:
import pickle
import re
import string
import pandas as pd
import numpy as np

from textblob import TextBlob
from collections import Counter

from langdetect import detect

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF, LatentDirichletAllocation

import nltk
from nltk.tokenize import TweetTokenizer, word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [2]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_colwidth', None)

In [3]:
# must uncomment & run the first time to DOWNLOAD NLTK data
# I used package identifier 'popular'
# nltk.download()

In [4]:
data = pd.read_pickle("pickle/balanced_nov2_tweets.pick")
data.shape

(96000, 9)

In [5]:
# keep only necessary columns
data['original'] = data.tweet
data.drop(columns='tweet', inplace=True)
data.head(3)

,id,date,time,user_id,username,hashtags,trump,biden,original
181142,1323379284434669568,2020-11-02,21:39:43,2820503362,artistacriseida,[],False,True,"All these articles showing that Biden is in the lead.... IGNORE THAT AND STILL GO VOTE. All of these maps showing information that may or may not be correct won’t matter on Election Day. Hillary was also in the lead last election, just do your part. ⁽ᶠᵘᶜᵏ ᵗʳᵘᵐᵖ⁾"
0,1323414585995526144,2020-11-02,23:59:59,1312487180258820096,annapieters17,[],False,True,@FoxNews Lady Gaga’s a nobody. Can’t figure out her own life and can’t even see nobody can help Biden. He’s out of the game from the day he gets in the game.
4,1323414585232293888,2020-11-02,23:59:59,2335763630,kylechwatt,[],False,True,"@The_Grupp “It is purely a fortuity that this isn’t one of the great mass casualty events in American history,” Ron Klain, who was Biden’s chief of staff at the time, said of H1N1 in 2019.” https://t.co/Umi317supK"


## Pre-Processing Pipeline

Now it's time to tokenize our tweets. Here are our pre-processing steps:
* Remove URLs
* Remove Twitter handles
* Remove numbers
* Convert to lowercase
* Remove punctuation
* Remove repeated letters so spell check will work ('aaaaand' -> 'aand')
* Remove non-English words
* Remove stop words

Since we're working with so many different words, I've chosen to use **lemmatization** instead of stemming for two reasons:
1. Lemmatization accurately reduces words to true meaning
2. Inxreased word reduction (handles synonyms)

In [6]:
# nltk.download() # must run first time (download 'popular')

In [7]:
# custom word dictionaries
from more_words import more_words as custom_words
from stop_words import stop_words as custom_stop_words
from bigrams import bigrams

import time
import nltk
from nltk.corpus import words, stopwords

def clean_tweet(tweet):
    """Pre-processing pipeline."""
    
    tweet = tweet.lower()
    tweet = re.sub(r"u\.s\. ", "usa", tweet)
    tweet = re.sub(r"https?:\/\/\S+", "", tweet)
    tweet = re.sub(r'\w*\d\w*', ' ', tweet)
    tweet = re.sub(r'\.{2,6}', ' ', tweet)
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    tweet = re.sub(r"([a-z])\1{2,5}", r'\1', tweet)
    tweet = ' '.join(tweet.split())
    
    for old, new in bigrams:
        tweet = re.sub(old, new, tweet) # ex: ('white house', 'whitehouse')
    
    return tweet

def tokenize(tweet, all_words, stop_words):
    """
    Returns all of the tokens in a cleaned tweet
    Parameters:
        - tweet (Series, required)
        - stop_words (set, required)
        - all_words (set, required) 
        
    Note:
        - any words not included in all_words here will be
          removed from tokens (including bigrams)
    """
    
    tweet = clean_tweet(tweet)
    
    twt = TweetTokenizer()    
    lemm = WordNetLemmatizer()  
    # lemmatize tokens & remove stop words
    tokens = [lemm.lemmatize(token) for token in twt.tokenize(tweet) if token not in stop_words]
    # only include words that are in our customized list of words
    tokens = [token for token in tokens if token in all_words]
    combined_tokens = ' '.join(tokens)

    return combined_tokens

def clean_and_tokenize(original_tweets):
    """
    Efficiently cleans, tokenizes, lemmatizes, and implements customized
    bigrams on a list of tweets.
    
    Parameters:
        original_tweets (required, pd.Series)
    
    Returns:
        cleaned, tokenized tweets (np.array)
    """
    tweet_array = original_tweets.to_numpy()

    all_words = list(words.words('en')) + custom_words
    stop_words = set(list(stopwords.words('english')) + custom_stop_words)
    
    for _, new in bigrams:
        all_words.append(new)

    all_words = set(all_words)
    print("num words: ", len(all_words))
    print("num stop words: ", len(stop_words))
    
    clean_tweets = [tokenize(tweet, all_words, stop_words) for tweet in tweet_array]
    
    return clean_tweets

In [8]:
%%time

data['tweet'] = clean_and_tokenize(data.original)
data.shape

num words:  236121
num stop words:  396
CPU times: user 40.3 s, sys: 111 ms, total: 40.4 s
Wall time: 40.4 s


(96000, 10)

In [9]:
data['num_tokens'] = data['tweet'].str.count(' ') + 1

mask = data['num_tokens'] >= 5
data_minlen = data[mask]
data_minlen.head()

,id,date,time,user_id,username,hashtags,trump,biden,original,tweet,num_tokens
181142,1323379284434669568,2020-11-02,21:39:43,2820503362,artistacriseida,[],False,True,"All these articles showing that Biden is in the lead.... IGNORE THAT AND STILL GO VOTE. All of these maps showing information that may or may not be correct won’t matter on Election Day. Hillary was also in the lead last election, just do your part. ⁽ᶠᵘᶜᵏ ᵗʳᵘᵐᵖ⁾",article showing lead ignore govote map showing information may may correct electionday hillaryclinton lead lastelection doyourpart,16
0,1323414585995526144,2020-11-02,23:59:59,1312487180258820096,annapieters17,[],False,True,@FoxNews Lady Gaga’s a nobody. Can’t figure out her own life and can’t even see nobody can help Biden. He’s out of the game from the day he gets in the game.,foxnews lady nobody figure life nobody help game get game,10
4,1323414585232293888,2020-11-02,23:59:59,2335763630,kylechwatt,[],False,True,"@The_Grupp “It is purely a fortuity that this isn’t one of the great mass casualty events in American history,” Ron Klain, who was Biden’s chief of staff at the time, said of H1N1 in 2019.” https://t.co/Umi317supK",purely fortuity mass casualty event history chief staff,8
5,1323414585097908224,2020-11-02,23:59:59,3282973915,medianpolitics,[],False,True,@MSNBC What? When words just don't match reality. BLM .com Antifa . com link to Biden. @ Billion worth of property damage. Innocent dinners being shouted at and forced to leave. Mobs roaming neighborhoods demanding you give them your house. This happened. Where were Dem's? https://t.co/TOMXgURZKb,msnbc word match reality blacklivesmatter link billion worth property innocent dinner forced leave mob roaming neighborhood demanding house,18
12,1323414582539493378,2020-11-02,23:59:58,1254928044529274883,justiceliberty0,[],False,True,@sap06_ky @PeteButtigieg @JoeBiden Let's start with welfare support and education. Show me one Republican who doesn't believe we should have any welfare at all or any education. That is an ignorant statement. Republicans believe in helping those in need through welfare and in education. Just because Republicans,let start welfare education doesnt believe welfare education ignorant statement republican believe helping welfare education republican,16


In [10]:
data_minlen.shape

(60588, 11)

In [11]:
mask = (data_minlen.trump == 0) & (data_minlen.biden == 1)
biden = data_minlen[mask]
mask = (data_minlen.trump ==1) & (data_minlen.biden == 0)
trump = data_minlen[mask]

df_limited = pd.concat([biden, trump])

print(f"\n Trump Tweets {len(trump)}\n\n Biden Tweets: {len(biden)}",)




 Trump Tweets 31997

 Biden Tweets: 28591


In [12]:
df_limited.shape

(60588, 11)

In [13]:
df_limited.to_pickle("pickle/n2tk_limited.pick")

In [14]:
data[['original', 'tweet']].sample(15)

,original,tweet
47109,"@BugLad12 @LetsGoToMarsppl @babybearclari @jschlatt If today's America were like the Weimar Republic (a failed state), Trump would make himself president for life. If it were possible he would do it",today republic life possible
64481,"Spadające notowania S&amp;P 500 w okresie przedwyborczym świadczą o tym, że większe szanse w #wyboryUSA ma kandydat demokratów Joe Biden. https://t.co/EBkYXr7Ygs",w w
255,@nbHolidayDoc Yup. All my friends but one voted for Trump as well as my whole family except one cousin.,friend whole family except cousin
60711,"@claudia93906 @cher That's because the con game Biden, the Democrats and the radicals in the news media have you in and your not aware that your being conned. I didn't realize it either for the longest time most of my life actually as well as all of your life and the lives of most Americans. We are",game democrat radical news medium aware realize either life life life
71139,@senadorhumberto #Trump #BolsonaroOrgulhoDoBrasil 🇺🇲🇧🇷 https://t.co/tuuir2J39t,
...,...,...
12729,Fuck this...when the hell did we ever need to board up Santa Monica for a general prez election??? This is Trumps game &amp; fault...please vote for the soul of our country &amp; democracy 🙏🤷‍♀️💯 they lie all day everyday and it is dangerous,hell board general trump game fault vote soul democracy lie everyday dangerous
7302,"Gli attentatori hanno dovuto anticipare la strage per il #lockdown. Avrebbero voluto farlo dopo le #Election2020 per non favorire #Trump, i guess. #quartarepubblica",lockdown non guess
80137,"I’m attending 2020 Victory’s event, “Swing Left Sonoma County Election Day Biden-Harris Victory Phone Bank” – sign up now to join me! https://t.co/BzxEw79qru",victory event swing left county electionday bidenharris victory phone bank sign join
44404,@svizzerams @DouglasEmhoff @JoeBiden @KamalaHarris @johnlegend You're welcome. I prefer Douglas though.,welcome prefer though


In [15]:
# data.to_pickle("pickle/tweets_df_5000tw.pick")

In [16]:
sample = 'tweet tokenize me please mr. biden helloaskldjalksfj I  pence    voting rights am asking for a favor continuous breakdown American Americans'

clean_and_tokenize(pd.Series(sample))


num words:  236121
num stop words:  396


['tweet votingrights continuous breakdown']

In [17]:
lemm = WordNetLemmatizer()
lemm.lemmatize('organizations')

'organization'

In [18]:
mask = len([word_tokenize(token) for token in data.tweet])

In [19]:
# data.to_pickle("pickle/n2_tokenized_eff_min3.pick")